In [1]:
import os.path as osp
from tqdm.auto import tqdm
import numpy as np
import wandb

import torch
from sklearn.metrics import roc_auc_score

import torch.nn.functional as F
# from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
from torch.nn import LayerNorm, Linear, ReLU, Tanh
from torchvision.ops import MLP
from torchvision.utils import save_image


import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv, GatedGraphConv, SAGEConv, GAT
from torch_geometric.nn import DeepGCNLayer, GENConv
from torch_geometric.utils import negative_sampling, to_dense_adj
from torch_geometric.loader import DataLoader

from dataset_processing4 import RNADataset


/home/vdshk/SecondaryStructurePredictionGNN/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dataset = RNADataset(root="./data4/")
dataset = dataset.shuffle()
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]
bs = 1

train_dataloader = DataLoader(train_data, batch_size=bs, shuffle=False)
val_dataloader = DataLoader(val_data, batch_size=1, shuffle=False)

In [3]:
train_data[0]

Data(x=[551, 8], edge_index=[2, 24140], y=[551], s=54)

In [4]:
dataset = list(filter(lambda x: x.s.item() <= 150, dataset))

In [5]:
dataset

[Data(x=[551, 8], edge_index=[2, 24140], y=[551], s=54),
 Data(x=[1698, 8], edge_index=[2, 132844], y=[1698], s=91),
 Data(x=[334, 8], edge_index=[2, 11008], y=[334], s=48),
 Data(x=[1092, 8], edge_index=[2, 68202], y=[1092], s=79),
 Data(x=[2761, 8], edge_index=[2, 272626], y=[2761], s=120),
 Data(x=[4274, 8], edge_index=[2, 530432], y=[4274], s=150),
 Data(x=[2850, 8], edge_index=[2, 286786], y=[2850], s=120),
 Data(x=[1185, 8], edge_index=[2, 76338], y=[1185], s=78),
 Data(x=[96, 8], edge_index=[2, 1596], y=[96], s=22),
 Data(x=[1969, 8], edge_index=[2, 164934], y=[1969], s=98),
 Data(x=[1829, 8], edge_index=[2, 150110], y=[1829], s=99),
 Data(x=[2533, 8], edge_index=[2, 240496], y=[2533], s=117),
 Data(x=[21, 8], edge_index=[2, 148], y=[21], s=10),
 Data(x=[155, 8], edge_index=[2, 3426], y=[155], s=29),
 Data(x=[626, 8], edge_index=[2, 28908], y=[626], s=56),
 Data(x=[578, 8], edge_index=[2, 25996], y=[578], s=54),
 Data(x=[1994, 8], edge_index=[2, 167354], y=[1994], s=100),
 Data(

In [6]:
train_data, val_data, test_data = dataset[0:655], dataset[655:873], dataset[873:]

In [139]:
def del_nucl_conn(y_pr, y_tr):    
    y_tr_local = torch.clone(y_tr).to(device)
    y_tr_local[(y_tr_local == 1)] = 0
    y_pr = y_pr + y_tr_local
    y_pr[(y_pr < 0)] = 0
    y_tr_local1 = torch.clone(y_tr)
    y_tr_local1[(y_tr_local1 == -1)] = 0
    return y_pr, y_tr_local1.to(device)


def del_main_diag(y):
    main_diag = torch.ones([1, y.size()[0]])
    return y * (torch.diag_embed(main_diag).to(device) - 1)[0] * (-1)


def adj_mat_split(y):
    y = list(torch.split(y, 196, dim=0))
    for i in range(len(y)):
        y[i] = torch.split(y[i], 196, dim=1)
    
    y1 = []
    for i in range(len(y)):
        y1.append(argm(y[i][i]))
    return torch.cat(y1, 0).to(device)
    

def argm(y):
    max_ind = torch.argmax(y, dim=1)
    y1 = torch.zeros_like(y)
    k = 0
    for i in max_ind:
        y1[k][i] = 1.
        y1[i][k] = 1.
        k += 1
    
    return y1 * y

def del_nucl_conn1(y_pred, y_true):
    y_tr_local = torch.clone(y_true)
    y_tr_local[(y_tr_local == [0, 1])] = torch.tensor([0, 0])
    y_tr_local[(y_tr_local == [1, 0])] = torch.tensor([0, 0])
    y_pred = y_pred + y_tr_local.to(device)
    y_pred_local = torch.tensor(list(map((lambda x: [0, 0] if x[0] < 0 else [1, 1]), y_pred))).to(device)
    y_pred = y_pred * y_pred_local
#     y_pred[torch.min(y_pred) < 0.] = torch.tensor([0, 0])
        
    #y_pred[(y_pred[1] < 0)] = 0

    y_tr_local1 = torch.clone(y_true)
    y_tr_local1[(y_tr_local1 == [-1, -1])] = torch.tensor([0, 0])
    return y_pred, y_tr_local1.to(device)



In [147]:
a = 0.5
beta = 1
def precision(y_pred, y_true):
    y_pred = torch.tensor(list(map((lambda x: 0 if x[0] > x[1] else 1), y_pred))).to(device)
    y_pred, y_true = del_nucl_conn(y_pred, y_true)
    #y_pred[(y_pred > a)] = 1
    #y_pred[(y_pred <= a)] = 0 
#     y_pred = argm(y_pred)
    tp = torch.sum(y_pred * y_true)
    fp = torch.sum((1 - y_true) * y_pred)
    
    return tp / (tp + fp + epsilon)

def recall(y_pred, y_true):    
    y_pred = torch.tensor(list(map((lambda x: 0 if x[0] > x[1] else 1), y_pred))).to(device)

    y_pred, y_true = del_nucl_conn(y_pred, y_true)
    #y_pred[(y_pred > a)] = 1
    #y_pred[(y_pred <= a)] = 0 
#     y_pred = argm(y_pred)
#     y_pred = adj_mat_split(y_pred)

    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    
    return tp / (tp + fn + epsilon)

def f_loss(y_pred, y_true):
    y_true_local = torch.tensor(list(map((lambda x: [1, 0] if x == 1 else ([0, 1] if x == 0 else [-1, -1])))))
    y_pred, y_true = del_nucl_conn1(y_pred, y_true_local)
#     y_pred, y_true = del_nucl_conn(y_pred, y_true)    
    tp = torch.sum(y_pred * y_true)
    fn = torch.sum(y_true * (1 - y_pred))
    fp = torch.sum((1 - y_true) * y_pred)
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)

    k1 = 1 - torch.abs(precision - recall)

    f1 = ((1 + beta * beta) * precision * recall) / (beta * beta * precision + recall + epsilon)
    
    
    # dp = abs(torch.mean(y_pred) - torch.mean(y_true))

    return (1 - f1)
#     return dp

def mse_loss(y_pred, y_true):
    y_true_local = torch.tensor(list(map((lambda x: [1, 0] if x == 1 else ([0, 1] if x == 0 else [-1, -1])), y_true)))
    y_pred, y_true = del_nucl_conn1(y_pred, y_true_local)
#     y_pred, y_true = del_nucl_conn(y_pred, y_true)
    
    return torch.mean((y_pred - y_true) ** 2)

In [148]:
class DeepGCNModel(torch.nn.Module):
    def __init__(self, num_blocks, hidden_channels, num_layers, dr, mlp):
        super(DeepGCNModel, self).__init__()
        self.layers = torch.nn.ModuleList()
        gat1 = GAT(8, hidden_channels, num_layers=num_layers)
        layer = DeepGCNLayer(gat1, block='res', act=torch.relu, dropout=dr)
        self.layers.append(layer)
        for i in range(1, num_blocks):
            conv = GAT(hidden_channels, hidden_channels, num_layers=num_layers, act=torch.relu)            

            layer = DeepGCNLayer(conv, block='res', act=torch.relu, dropout=dr)
            self.layers.append(layer)
            
            
        self.mlp = MLP(hidden_channels, mlp, dropout=0)

        
    def forward(self, x, edge_index):
        x = self.layers[0].conv(x, edge_index)
        for layer in self.layers[1:]:
            x = layer(x, edge_index)
        
        x = self.mlp(x)
        
        return torch.softmax(x, dim=0)


In [149]:
num_blocks = 7
num_layers = 2
hidden_channels = 70

dr = 0.15


lr = 0.00001
epochs = 150000
mlp = [20, 10, 2]


model = DeepGCNModel(num_blocks, hidden_channels, num_layers, dr,mlp)
# model = torch.load("./models1/ResGATConv__MSE_mlp_[200, 150, 100, 50, 10, 1]_1*2_0.0001_Adam_250_0.05_0.2_1_1500.pt")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
train_data = list(map(lambda x: x.to(device), train_data))
val_data = list(map(lambda x: x.to(device), val_data))
print(device)

# optimizer = torch.optim.Adam(model.parameters(), lr=lr)
# optimizer = torch.optim.Adagrad(model.parameters(), lr=lr)
optimizer = torch.optim.RMSprop(model.parameters(), lr=lr)
#optimizer = torch.optim.Adamax(model.parameters(), lr=lr)
# optimizer = torch.optim.Adadelta(model.parameters(), lr=lr)

# optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

# criterion = torch.nn.BCELoss()
# criterion = torch.nn.BCEWithLogitsLoss()
# criterion = torch.nn.CrossEntropyLoss()
# criterion = torch.nn.KLDivLoss()

def RMSELoss(y_pred, y_true):
    y_pred, y_true = del_nucl_conn(y_pred, y_true)

    return torch.sqrt(torch.sum((y_pred - y_true) ** 2))

criterion = mse_loss
# criterion = RMSELoss
#criterion = f_loss

epsilon = 1e-30

cuda


In [150]:
model

DeepGCNModel(
  (layers): ModuleList(
    (0-6): 7 x DeepGCNLayer(block=res)
  )
  (mlp): MLP(
    (0): Linear(in_features=70, out_features=20, bias=True)
    (1): ReLU()
    (2): Dropout(p=0, inplace=False)
    (3): Linear(in_features=20, out_features=10, bias=True)
    (4): ReLU()
    (5): Dropout(p=0, inplace=False)
    (6): Linear(in_features=10, out_features=2, bias=True)
    (7): Dropout(p=0, inplace=False)
  )
)

In [151]:
run_name = f'ResGATConv_softmax_mse_relu_mlp_{mlp}_{num_blocks}*{num_layers}_{lr}_RMSProp_{hidden_channels}_{a}_{dr}_{beta}'


wandb.init(
    # set the wandb project where this run will be logged
    project="secondary_structure_prediction1",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "architecture": "ResGATConv",
    "epochs": epochs,
    "optimizer": "RMSProp",
    "out_channels": hidden_channels,
    "loss": "mse",
    "beta": beta,
    "train:val:test": "655:218:218",
    "dataset": 4
    },
    name=run_name
)



In [152]:
def train():
    for epoch in range(1, epochs + 1):
        model.train()
        train_loss = []
        train_recall = []
        train_precision = []
        for g in tqdm(train_data):

            optimizer.zero_grad()

            out = model(g.x, g.edge_index)
            y_true = g.y.unsqueeze(dim=1)
#             print(out)
            loss = criterion(out, y_true)            
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
            train_precision.append(precision(out, y_true).item())
            train_recall.append(recall(out, y_true).item())
            
        train_prec = np.mean(train_precision)
        train_rec = np.mean(train_recall)
        train_f1 = (2 * train_prec * train_rec) / (train_prec + train_rec)
        print(f'Epoch: {epoch:03d}, loss: {np.mean(train_loss)}, f1: {train_f1}, precision: {train_prec}, recall: {train_rec}')
        
        val_loss = []
        val_recall = []
        val_precision = []
        with torch.no_grad():
            for g in val_data:
                g.to(device)
                out = model(g.x, g.edge_index)
        
                y_true = g.y.unsqueeze(dim=1)
                loss = criterion(out, y_true)
                                
                val_loss.append(loss.item())
                                
                val_precision.append(precision(out, y_true).item())
                val_recall.append(recall(out, y_true).item())
 
            
            prec = np.mean(val_precision)
            rec = np.mean(val_recall)
            f1 = (2 * prec * rec) / (prec + rec)
            print(f'val_loss: {np.mean(val_loss)}, val_f1: {f1}, val_precision: {prec}, val_recall: {rec}')
              
            wandb.log({"train_loss": np.mean(train_loss), "train_f1": train_f1, "train_precision": train_prec, 
                       "train_recall": train_rec,
                       "val_loss": np.mean(val_loss), "val_f1": f1, "val_precision": prec, "val_recall": rec})
#     wandb.finish()

In [ ]:
train()
torch.save(model, "./models1/" + run_name + "_" + str(epochs) + ".pt")

100%|█████████████████████████████████████████| 655/655 [02:21<00:00,  4.63it/s]


Epoch: 001, loss: 0.11681123857052271, f1: 0.08730639081076975, precision: 0.04784781389109051, recall: 0.49794992718077796
val_loss: 0.12313142417152541, val_f1: 0.08979900393161871, val_precision: 0.04936308726463296, val_recall: 0.49654658425838577


100%|█████████████████████████████████████████| 655/655 [02:21<00:00,  4.64it/s]


Epoch: 002, loss: 0.11681124341624384, f1: 0.08732746848758954, precision: 0.04784781389109051, recall: 0.49932467973869266
val_loss: 0.12313141574690101, val_f1: 0.08980805963262839, val_precision: 0.04936308726463296, val_recall: 0.49710091606739465


100%|█████████████████████████████████████████| 655/655 [02:20<00:00,  4.65it/s]


...

100%|█████████████████████████████████████████| 655/655 [02:20<00:00,  4.67it/s]


Epoch: 314, loss: 0.11680126787938235, f1: 0.08541499307732077, precision: 0.04784781389109051, recall: 0.39753583408039034
val_loss: 0.12312302596593669, val_f1: 0.08741711274384707, val_precision: 0.04936308726463296, val_recall: 0.3815682190684004


100%|█████████████████████████████████████████| 655/655 [02:20<00:00,  4.67it/s]


Epoch: 315, loss: 0.11680071413630748, f1: 0.08509327940647274, precision: 0.04784781389109051, recall: 0.3840212802054318
val_loss: 0.12312286131835859, val_f1: 0.0868610435572915, val_precision: 0.04936308726463296, val_recall: 0.361372266614109


 75%|██████████████████████████████▋          | 490/655 [01:46<00:45,  3.62it/s]

In [ ]:
torch.save(model, "./models1/" + run_name + "_" + str(22) + ".pt")

In [87]:
wandb.finish()